# Using Pandas dataframes for relational database functionality

Some basic terminology:
- Python is the programming *language* used in Jupyter notebooks.
- Jupyter notebooks is the cell-based *app* that implements Python.
- Pandas is a *library* of functions designed to work especially with tabular data.

In [1]:
# first import pandas and numpy to make sure the relevant functionality is available to us

import numpy as np
import pandas as pd

from pandas import DataFrame, Series

Now we need to *read in* some data to work with. The base data is usually in a CSV file on your hard drive (though you can read in other kinds of files, and it doesn't necessarily need to be stored on your personal hard drive). As of this cell, there is no data stored in local memory. We need to tell Python where to find our data and to save it as a local variable in this notebook. To do that, we need to know its *path*, i.e. its hierarchical location in the computer file system.

Remember, two dots (`..`) tells the path to jump up one directory.

To start, I am going to read in data for my own personalized dictionary of terminology useful and specific to my research.

In [2]:
# read in glossary data

glossary = pd.read_csv('../../DropBox/Active_Directories/Digital_Humanities/Datasets/exported_database_data/glossary.csv', \
                       names=['UID', 'Term', 'Emic_Term', 'Translit', 'Scope', 'Tags'])

We have now saved our CSV data as a *dataframe* variable called `glossary`. We can look at the entire thing simply by calling `glossary`. However, quite often our tables will be quite large, which means it might be easier to get some descriptive information about what we are dealing with.

In [8]:
glossary.head(5)

#glossary.describe(include='all')

,UID,Term,Emic_Term,Translit,Scope,Tags
0,1,qonalgha,قونالغه,NaN,transoxania khorezm,military
1,2,jarib,جریب,NaN,transoxania,agriculture measurement
2,3,tanabana,طنابانه,ṭanābāna tạnāp,transoxania,taxes
3,4,tarkhan,ترخان درخان,tarkhān,khorezm transoxania,taxes property
4,5,yap,جیب,NaN,khorezm transoxania,irrigation


This is all well in good, but so far not really much better than just opening the CSV file in Excel. And, wait a second: where is the column for the most important piece of information in a dictionary, i.e. the definition!

Many words have different definitions depending on the time period and context. This tells you that, in relational database terms, we are dealing with a *one to many* relationship, which cannot be captured by a single table.

What we need is a separate table of definitions with a *join key* that corresponds to the unique id (UID) in the glossary table. So let's read in another table.

In [9]:
# read in definition data

definitions = pd.read_csv('../../DropBox/Active_Directories/Digital_Humanities/Datasets/exported_database_data/definitions.csv', \
                       names=['DefID', 'JoinKey', 'Def', 'Spec', 'Source'])



In [10]:
definitions.head(10)

,DefID,JoinKey,Def,Spec,Source
0,1,3,"This tax was ""calculated on the basis of the c...",ferghana,280.0
1,2,14,"""generally corresponded to a one-fifth share o...",ferghana,280.0
2,3,15,"""The first [kind of crop] was collected on ti...",ferghana,280.0
3,4,3,"""Tạnāp was reportedly paid on orchards, gard...",ferghana,280.0
4,5,16,"""In the countryside, livestock was the basis t...",ferghana,280.0
5,6,17,"""the khan’s land constituted a distinct catego...",ferghana,280.0
6,7,18,"""the khan’s land constituted a distinct catego...",ferghana,280.0
7,8,19,"""other plots that were privately owned by the ...",ferghana,289.0
8,9,20,"""some of them were exempted from land-tax (āq...",ferghana,280.0
9,10,21,"""some of them were exempted from land-tax (āq...",ferghana,280.0


So now we have all of the raw data we have for a nice dictionary that offers multiple definitions for a single term, and even with some nice additional data about the context and source of that particular definition. But, currently, the computer has no idea that these two dataframes are related to one another. We need to change that.


In [11]:
merge = pd.merge(glossary, definitions, left_on="UID", right_on="JoinKey")


In [13]:
merge.head(5)

,UID,Term,Emic_Term,Translit,Scope,Tags,DefID,JoinKey,Def,Spec,Source
0,1,qonalgha,قونالغه,NaN,transoxania khorezm,military,623,1,A term found in documents thoughout the Turko-...,NaN,NaN
1,1,qonalgha,قونالغه,NaN,transoxania khorezm,military,624,1,"""This Turkish word which appears in the Farman...",NaN,983.0
2,2,jarib,جریب,NaN,transoxania,agriculture measurement,625,2,"""Jarīb was the former land measure, also the m...",NaN,983.0
3,2,jarib,جریب,NaN,transoxania,agriculture measurement,626,2,"A unit of measure, both of volume and of land....",NaN,NaN
4,3,tanabana,طنابانه,ṭanābāna tạnāp,transoxania,taxes,1,3,"This tax was ""calculated on the basis of the c...",ferghana,280.0


Already this is pretty useful: a single table that combines information from both of our tables, but much more efficiently and flexibly than if we had inputted the data in this format to begin with.

Now let's clean up the table a bit by eliminating some of the duplicate columns.

In [14]:
# drop duplicate columns
merge = merge.drop('DefID', axis=1)

# rename columns
#merge.rename(columns = {'UID_y':'UID'}, inplace = True)


In [15]:
merge.head(10)

,UID,Term,Emic_Term,Translit,Scope,Tags,JoinKey,Def,Spec,Source
0,1,qonalgha,قونالغه,NaN,transoxania khorezm,military,1,A term found in documents thoughout the Turko-...,NaN,NaN
1,1,qonalgha,قونالغه,NaN,transoxania khorezm,military,1,"""This Turkish word which appears in the Farman...",NaN,983.0
2,2,jarib,جریب,NaN,transoxania,agriculture measurement,2,"""Jarīb was the former land measure, also the m...",NaN,983.0
3,2,jarib,جریب,NaN,transoxania,agriculture measurement,2,"A unit of measure, both of volume and of land....",NaN,NaN
4,3,tanabana,طنابانه,ṭanābāna tạnāp,transoxania,taxes,3,"This tax was ""calculated on the basis of the c...",ferghana,280.0
5,3,tanabana,طنابانه,ṭanābāna tạnāp,transoxania,taxes,3,"""Tạnāp was reportedly paid on orchards, gard...",ferghana,280.0
6,3,tanabana,طنابانه,ṭanābāna tạnāp,transoxania,taxes,3,ماليات بر باغها و تاكستانها از هر طناپ مربع در...,transoxania,283.0
7,3,tanabana,طنابانه,ṭanābāna tạnāp,transoxania,taxes,3,report comparing khiraj and tanabana in Bukhar...,bukhara,299.0
8,4,tarkhan,ترخان درخان,tarkhān,khorezm transoxania,taxes property,4,واژهاي كهن؛ امتياز معافيت ماليـاتي، بـراي يـك ...,NaN,283.0
9,5,yap,جیب,NaN,khorezm transoxania,irrigation,5,Artificial trench for irrigation.,khorezm,273.0


There is a separate table with information about the Source a given definition came from: we can add that later using the same `merge` method we used to join the terms to their definitions.

But for now let's implement some basic sorting and searching functionality. Say we want to search for all of the different definitions of the word 'mahzar':

In [16]:
query_mask = merge["Def"].str.contains("document", na=False)

In [17]:
results = merge[query_mask]


In [18]:
results

,UID,Term,Emic_Term,Translit,Scope,Tags,JoinKey,Def,Spec,Source
0,1,qonalgha,قونالغه,NaN,transoxania khorezm,military,1,A term found in documents thoughout the Turko-...,NaN,NaN
19,10,mutabiq bi-aslihi,مطابق باصله,muṭābiq bi-aṣlihi,NaN,NaN,10,"glossed as the Latin ""vidimus"": an official or...",NaN,NaN
22,12,bigha,بيگها بیگهه बीघा,NaN,indic mughal,measurement,12,This term active in revenue documents during A...,NaN,519.0
42,17,quruq,قرق قوروق,qūrūq,transoxania,agriculture property,17,In Mughal documents can refer to removing some...,mughals,NaN
58,27,farman,فرمان,NaN,persianate,royal decree superior_to_junior,27,A crucial element is that the farman signifies...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
616,419,nawazish-nama,نوازشنامه,NaN,NaN,document_type,419,used to refer to a document sent by Qushbegi t...,bukhara,1060.0
618,420,jawf,جوف,NaN,NaN,NaN,420,this term is often used in Bukharan documents ...,bukhara,1064.0
642,437,tala,تله,NaN,NaN,NaN,437,usage paired with تالان confirms this usage in...,bukhara,1220.0
662,450,lahaz,لاحظ,NaN,NaN,illness,450,1921 BNSR Turki document: هر ئیل ده یاز کونلار...,bukhara,1366.0


In [19]:
# change dispaly options so we can see the full definition
pd.set_option('display.max_colwidth', None)

In [20]:
results

,UID,Term,Emic_Term,Translit,Scope,Tags,JoinKey,Def,Spec,Source
0,1,qonalgha,قونالغه,NaN,transoxania khorezm,military,1,"A term found in documents thoughout the Turko-Mongol world in various forms, all of which are derived from qonmoq. It refers to the obligation of the rural populace to billet state officials and military personnel in their homes for a rest or for the night, providing them with food and entertainment, as well as provisions for the next day's journey. The billeting duty is described as one of the heaviest obligations imposed on the common people, even being referred to as a 'national disaster' in Ilkhanid Iran.",NaN,NaN
19,10,mutabiq bi-aslihi,مطابق باصله,muṭābiq bi-aṣlihi,NaN,NaN,10,"glossed as the Latin ""vidimus"": an official or legal inspection (as of a document); an attested copy of a document",NaN,NaN
22,12,bigha,بيگها بیگهه बीघा,NaN,indic mughal,measurement,12,"This term active in revenue documents during Akbar's time, e.g. in year 979/1571.",NaN,519.0
42,17,quruq,قرق قوروق,qūrūq,transoxania,agriculture property,17,In Mughal documents can refer to removing someone from office.,mughals,NaN
58,27,farman,فرمان,NaN,persianate,royal decree superior_to_junior,27,"A crucial element is that the farman signifies a degree of sovereignty: ""A farman is a public and legislative document promulgated in the name of the ruler or another person (e.g. prince, princess, governor) holding partial elements of sovereignty.""",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
616,419,nawazish-nama,نوازشنامه,NaN,NaN,document_type,419,used to refer to a document sent by Qushbegi to Kulab gov. in 1913,bukhara,1060.0
618,420,jawf,جوف,NaN,NaN,NaN,420,"this term is often used in Bukharan documents to refer to an 'enclosed' document, e.g.: بجوف خط کوپیۀ تفتیش شهر سمرقند",bukhara,1064.0
642,437,tala,تله,NaN,NaN,NaN,437,usage paired with تالان confirms this usage in Central Asian documents,bukhara,1220.0
662,450,lahaz,لاحظ,NaN,NaN,illness,450,1921 BNSR Turki document: هر ئیل ده یاز کونلارینده قطعی صوردتده مخالفِ مزاج انسانیّت اولدیغی خصوصنده انواع کسل لیک و انحاء لاحظ و بیمارلیک لر خلاوق ایچنده پیدا و ظاهر اُولُوب,bukhara,1366.0


Note that Dataframes can read *regular expressions*, a concept we have already encountered. So say you remember that there was a word that started with a 't', and ended with a 'gha', but you weren't sure which vowel came in between. The regular expression for that would be `t.*gha`.
- `.` means 'any character will do'
- `*` means 'whatever the previous character was, there can be as many of those until we hit the next search term (which is `gha` in this case)

In [37]:
query_mask = merge["Term"].str.contains("t.*gha", na=False)
results = merge[query_mask]
results

,UID,Term,Emic_Term,Translit,Scope,Tags,JoinKey,Def,Spec,Source
225,128,istighasa,NaN,NaN,indic,legal,128,"""Demanding justice, preferring a complaint.""",NaN,508.0
226,128,istighasa,NaN,NaN,indic,legal,128,"Arabic term means ""appeal for aid.""",NaN,NaN
315,195,tamgha,تمغا,NaN,transoxania,taxes signature seal,195,﻿Customs duties and commercial taxes; frequently used to denote all taxes contrary to the shari'a and therefore was often the target of pious Muslims seeking to abrogate such non-canonical levies.,NaN,NaN
316,195,tamgha,تمغا,NaN,transoxania,taxes signature seal,195,"﻿An abstract seal or stamp used by Eurasian nomadic peoples and by cultures influenced by them. The tamga was normally the emblem of a particular tribe, clan or family. They were common among the Eurasian nomads throughout Classical Antiquity and the Middle Ages (including Alans, Mongols, Sarmatians, Scythians and Turkic peoples).",NaN,NaN
317,195,tamgha,تمغا,NaN,transoxania,taxes signature seal,195,"""A word of foreign importation, it came to mean in India as a grant under the red seal of the Emperor, or to which red ink was applied... Such land assignments were reserved for an officer who applied for a grant as a state pension in his own home village (ba-jihat-i-vaṭan) in which he was born or desired to settle down.""",mughals,983.0
318,195,tamgha,تمغا,NaN,transoxania,taxes signature seal,195,"example of word being used in Russian to indicate a mark (in this case an 'X') in place of a signature at the end of a document : ""К сему прошению простел Умир Матьханов приложил тамгу X"". i.e. the document was written by a scribe fluent in Russian, but the petitioner was illiterate.",bukhara,1263.0
319,195,tamgha,تمغا,NaN,transoxania,taxes signature seal,195,Turki formula for using a symbol in place of a signature: شول سوزیمنی راست لیغینه قولوم قیوب تمخم باسدوم,russian_turkestan,1135.0


ah, but we didn't tell it that the term we are looking for necessarily needs to *start* with `t`; we can fix that usuing regex with `^`, which anchors the character that follows it as the first character of the instance.

In [38]:
query_mask = merge["Term"].str.contains("^t.*gha", na=False)
results = merge[query_mask]
results

,UID,Term,Emic_Term,Translit,Scope,Tags,JoinKey,Def,Spec,Source
315,195,tamgha,تمغا,NaN,transoxania,taxes signature seal,195,﻿Customs duties and commercial taxes; frequently used to denote all taxes contrary to the shari'a and therefore was often the target of pious Muslims seeking to abrogate such non-canonical levies.,NaN,NaN
316,195,tamgha,تمغا,NaN,transoxania,taxes signature seal,195,"﻿An abstract seal or stamp used by Eurasian nomadic peoples and by cultures influenced by them. The tamga was normally the emblem of a particular tribe, clan or family. They were common among the Eurasian nomads throughout Classical Antiquity and the Middle Ages (including Alans, Mongols, Sarmatians, Scythians and Turkic peoples).",NaN,NaN
317,195,tamgha,تمغا,NaN,transoxania,taxes signature seal,195,"""A word of foreign importation, it came to mean in India as a grant under the red seal of the Emperor, or to which red ink was applied... Such land assignments were reserved for an officer who applied for a grant as a state pension in his own home village (ba-jihat-i-vaṭan) in which he was born or desired to settle down.""",mughals,983.0
318,195,tamgha,تمغا,NaN,transoxania,taxes signature seal,195,"example of word being used in Russian to indicate a mark (in this case an 'X') in place of a signature at the end of a document : ""К сему прошению простел Умир Матьханов приложил тамгу X"". i.e. the document was written by a scribe fluent in Russian, but the petitioner was illiterate.",bukhara,1263.0
319,195,tamgha,تمغا,NaN,transoxania,taxes signature seal,195,Turki formula for using a symbol in place of a signature: شول سوزیمنی راست لیغینه قولوم قیوب تمخم باسدوم,russian_turkestan,1135.0


### Where to go from here?

One of the wonderful things about opensource data formats is that you are not limited to any one single application. A minimalist use case of Pandas would be to use its joining functionality to produce various permutations of different merges as your needs dictate and then export those merged tables back to CSV to use in other programs (e.g., Excel). Or you can run your searches within the Jupyter notebook, as we did above. Or you could write functions to create interactive searches on your command line - which is easier than it sounds.

As we will see in future weeks (if we have time), Python has other libraries that can accomplish other tasks. For instance, if you were tracking social network relationships in your database, you could use the above methods to get your data in the right format and subset that you need, and then visualize those relationships graphically right within your Jupyter notebook. Ditto for geospatial analysis.

But, for now, let's get this merged table back into a CSV file just so that we know how to do so (it is currently saved in a local dataframe variable within this instance of Jupyter notebook). *Caution*: you would not want to modify data after the export of the merged table; it is only really for reference and viewing. Your 'base' data should be those original CSV files. Remember, you can run this entire routine in an instance since it is code, so there is no real drawback to having separate tables for analysis and data entry.



In [45]:
merge.to_csv("exported_data/sample.csv")
# note that tags on separate lines gets messed up in export